In [12]:
import numpy
import pandas as pd

In [13]:
df = pd.read_csv('../../data/raw/Sentences_AllAgree.txt', header=None, names=["text", "label"], encoding="latin1", sep="@")

In [14]:
df.describe()

,text,label
count,2264,2264
unique,2259,3
top,SSH Communications Security Corporation is hea...,neutral
freq,2,1391


In [15]:
df.dropna()
df["text"] = df["text"].str.strip()
df["label"] = df["label"].str.strip()

In [16]:
df.describe()

,text,label
count,2264,2264
unique,2259,3
top,SSH Communications Security Corporation is hea...,neutral
freq,2,1391


In [17]:
df = df[df["label"] != "neutral"]

In [18]:
df.describe()

,text,label
count,873,873
unique,873,2
top,"For the last quarter of 2010 , Componenta 's n...",positive
freq,1,570


In [19]:
label_map = {
    "positive": 1,
    "negative": -1
}

df["label"] = df["label"].map(label_map)

In [20]:
df.head()

,text,label
1,"For the last quarter of 2010 , Componenta 's n...",1
2,"In the third quarter of 2010 , net sales incre...",1
3,Operating profit rose to EUR 13.1 mn from EUR ...,1
4,"Operating profit totalled EUR 21.1 mn , up fro...",1
5,Finnish Talentum reports its operating profit ...,1
